# Cleaning File
Get the basic company information here. There are three main parts here.

1. Company addresses

2. Basic change information

3. Controls for the companies


First, just load the data and the relevant deflators.

In [8]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [9]:
defl            = pd.read_csv("../GDPDEF.csv")
defl['DATE']    = pd.to_datetime(defl['DATE'])
defl['year']    = defl['DATE'].dt.year

# for some reason the years are messed up but they take anything after 1971 to be in the 2000s

defl.loc[(defl.year > 2020), 'year'] = defl.year - 100
defl['qtr'] = defl['DATE'].dt.quarter.astype('float64')
defl.drop(columns = {'DATE','GDPDEF'},inplace = True)
defl.tail()

,inflator,year,qtr
291,1.012932,2019,4.0
292,1.009643,2020,1.0
293,1.014000,2020,2.0
294,1.004838,2020,3.0
295,1.000000,2020,4.0


In [10]:
file = "../../data/companyData/compustat2000s.csv"
compustat = pd.read_csv(file, encoding = 'unicode_escape').drop(columns = {'curncdq'})

# rename to clean this up a little bit
compustat.rename(columns = {'fyearq':'year',
                            'fqtr':  'qtr',
                            'chq':   'cash', 
                            'conm':  'companyName',
                            'atq':   'assets',
                            'niq':   'netIncome',
                            'revtq': 'totalRevenue',
                            'cogsq': 'costGoodsSold',
                            'invtq': 'totalInv',
                            'oibdpq': 'opInc_befDep',
                            'oiadpq': 'opInc_afDep',
                            'prccq':  'priceClose'
                           },
                 inplace = True)


# and merge in the deflators
compustat = compustat.merge(defl)
compustat.head()

,gvkey,datadate,year,qtr,indfmt,consol,popsrc,datafmt,tic,companyName,...,opInc_afDep,opInc_befDep,totalRevenue,costat,priceClose,add1,addzip,city,state,inflator
0,1004,19990228,1998,3.0,INDL,C,D,STD,AIR,AAR CORP,...,18.938,23.200,250.984,A,15.1250,"One AAR Place, 1100 North Wood Dale Road",60191,Wood Dale,IL,1.51804
1,1082,19990131,1998,3.0,INDL,C,D,STD,SERV.1,SERVIDYNE INC,...,0.795,1.562,27.942,I,6.8750,"1945 The Exchange, Suite 325",30339-2029,Atlanta,GA,1.51804
2,1244,19990228,1998,3.0,INDL,C,D,STD,ALCD.,ALCIDE CORP,...,-0.090,-0.074,2.280,I,15.5000,8561 154th Avenue North East,98052,Redmond,WA,1.51804
3,1258,19990228,1998,3.0,INDL,C,D,STD,CGYNQ,CAPCO ENERGY INC,...,0.802,0.803,0.809,I,NaN,"1800 West Loop South, Suite 1950",77027,Houston,TX,1.51804
4,1331,19990131,1998,3.0,INDL,C,D,STD,APNI,ALPINE GROUP INC,...,NaN,NaN,326.714,I,13.9375,One Meadowlands Plaza,07073,East Rutherford,NJ,1.51804


In [11]:
compustat.columns

Index(['gvkey', 'datadate', 'year', 'qtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'companyName', 'curcdq', 'datacqtr', 'datafqtr',
       'assets', 'cash', 'costGoodsSold', 'totalInv', 'netIncome',
       'opInc_afDep', 'opInc_befDep', 'totalRevenue', 'costat', 'priceClose',
       'add1', 'addzip', 'city', 'state', 'inflator'],
      dtype='object')

In [12]:
compustat.columns

sum(compustat.datacqtr != compustat.datafqtr)
compustat.shape

(1005795, 28)

In [13]:
toDeflate = ['assets','costGoodsSold', 'totalInv', 'netIncome', 'totalRevenue',
            'opInc_afDep', 'opInc_befDep','priceClose','cash']


for col in toDeflate:
    compustat[col] = compustat[col]*compustat.inflator

    
    
compustat.columns

Index(['gvkey', 'datadate', 'year', 'qtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'companyName', 'curcdq', 'datacqtr', 'datafqtr',
       'assets', 'cash', 'costGoodsSold', 'totalInv', 'netIncome',
       'opInc_afDep', 'opInc_befDep', 'totalRevenue', 'costat', 'priceClose',
       'add1', 'addzip', 'city', 'state', 'inflator'],
      dtype='object')

In [14]:
compustat.drop(columns=['indfmt','consol','popsrc','datafmt','tic','datacqtr','datafqtr','inflator'], inplace = True)


compustat.columns

Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq', 'assets',
       'cash', 'costGoodsSold', 'totalInv', 'netIncome', 'opInc_afDep',
       'opInc_befDep', 'totalRevenue', 'costat', 'priceClose', 'add1',
       'addzip', 'city', 'state'],
      dtype='object')

## 1. Basic Addresses

In [105]:
compustatAddresses = compustat[['year','gvkey','companyName','add1','city','state','addzip']]
compustatAddresses.to_csv("../../data/companyData/compustatAddresses.csv")

Get the addresses for all companies in the SC dataset.

In [137]:
c_links = pd.read_csv("../../data/companyData/compustatSCLinked.csv")

c_links['year'] = c_links.srcdate.astype('str').str.slice(0,4).astype('int64')

c_links = c_links[c_links.year > 1997]

relevant_gvkeys = c_links.gvkey.append(c_links.cgvkey).drop_duplicates()

print(c_links.head(),relevant_gvkeys.shape)

    gvkey                        conm  cgvkey                       cconm  \
70   1013  ADC TELECOMMUNICATIONS INC    2136  VERIZON COMMUNICATIONS INC   
71   1013  ADC TELECOMMUNICATIONS INC    2136  VERIZON COMMUNICATIONS INC   
72   1013  ADC TELECOMMUNICATIONS INC    2136  VERIZON COMMUNICATIONS INC   
73   1013  ADC TELECOMMUNICATIONS INC    2136  VERIZON COMMUNICATIONS INC   
74   1013  ADC TELECOMMUNICATIONS INC    2136  VERIZON COMMUNICATIONS INC   

                      cnms   srcdate  cid  sid    ctype   salecs     scusip  \
70  VERIZON COMMUNICATIONS  20021031   10    0  COMPANY  111.056  000886309   
71  VERIZON COMMUNICATIONS  20041031   13    0  COMPANY  104.312  000886309   
72  VERIZON COMMUNICATIONS  20051031   13    0  COMPANY  146.000  000886309   
73  VERIZON COMMUNICATIONS  20061031   13    0  COMPANY  205.000  000886309   
74  VERIZON COMMUNICATIONS  20071031   13    0  COMPANY  236.000  000886309   

      stic     ccusip ctic  year  
70  ADCT.1  92343V104   VZ 

In [138]:
print("number of customers: ", c_links.gvkey.unique().shape[0],
      "total companies (incl suppliers): ", relevant_gvkeys.shape[0])

number of customers:  5411 total companies (incl suppliers):  7788


In [139]:
compustatAddresses = compustatAddresses[(compustatAddresses.year < 2020) & \
                          compustatAddresses.gvkey.isin(relevant_gvkeys)]

print(compustatAddresses.head(), compustatAddresses.gvkey.unique().shape)

   year  gvkey               companyName  \
0  1998   1004                  AAR CORP   
1  1998   1082             SERVIDYNE INC   
2  1998   1244               ALCIDE CORP   
4  1998   1331          ALPINE GROUP INC   
5  1998   1562  AMERICAN SOFTWARE  -CL A   

                                       add1             city state      addzip  
0  One AAR Place, 1100 North Wood Dale Road        Wood Dale    IL       60191  
1              1945 The Exchange, Suite 325          Atlanta    GA  30339-2029  
2              8561 154th Avenue North East          Redmond    WA       98052  
4                     One Meadowlands Plaza  East Rutherford    NJ       07073  
5             470 East Paces Ferry Road, NE          Atlanta    GA       30305   (7271,)


In [140]:
chq = compustatAddresses[['gvkey','companyName','add1',
                   'city','state','addzip']].drop_duplicates()

In [141]:
chq.gvkey.unique().shape

(7271,)

We're starting with the compustat north america dataset. Not all of the HQs are in North America, so we can filter some of the information down to match with Infogroup.

In [142]:
canadian = ['ON', 'AB','QC', 'BC', 'NS', 'NF', 'SK', 'MB', 'NB']

chq.state.unique()

array(['IL', 'GA', 'WA', 'NJ', 'PA', 'IN', 'OH', 'NC', 'OR', 'CO', 'TN',
       'CT', 'CA', 'NY', 'MN', 'MI', 'TX', 'MO', nan, 'MD', 'VA', 'LA',
       'NE', 'MA', 'KS', 'AR', 'AL', 'ON', 'OK', 'WI', 'FL', 'BC', 'AB',
       'AZ', 'MS', 'QC', 'UT', 'ND', 'NS', 'SC', 'ID', 'NV', 'KY', 'DE',
       'SD', 'NH', 'RI', 'IA', 'NB', 'ME', 'PR', 'HI', 'VT', 'DC', 'WV',
       'MT', 'NF', 'SK', 'WY', 'MB', 'NM', 'VI'], dtype=object)

In [143]:
chq = chq[~(chq.state.isin(canadian)) & ~chq.state.isna()]

In [144]:
chq.head()

,gvkey,companyName,add1,city,state,addzip
0,1004,AAR CORP,"One AAR Place, 1100 North Wood Dale Road",Wood Dale,IL,60191
1,1082,SERVIDYNE INC,"1945 The Exchange, Suite 325",Atlanta,GA,30339-2029
2,1244,ALCIDE CORP,8561 154th Avenue North East,Redmond,WA,98052
4,1331,ALPINE GROUP INC,One Meadowlands Plaza,East Rutherford,NJ,07073
5,1562,AMERICAN SOFTWARE -CL A,"470 East Paces Ferry Road, NE",Atlanta,GA,30305


In [145]:
chq.addzip.str.len().value_counts()

5.0     5488
10.0     761
Name: addzip, dtype: int64

In [146]:
chq['addzip'] = chq.addzip.astype('str').str.slice(0,5)

In [147]:
chq.rename(columns = {'conm': 'company','addzip': 'cstatZipcode'},inplace = True)
chq.head()

,gvkey,companyName,add1,city,state,cstatZipcode
0,1004,AAR CORP,"One AAR Place, 1100 North Wood Dale Road",Wood Dale,IL,60191
1,1082,SERVIDYNE INC,"1945 The Exchange, Suite 325",Atlanta,GA,30339
2,1244,ALCIDE CORP,8561 154th Avenue North East,Redmond,WA,98052
4,1331,ALPINE GROUP INC,One Meadowlands Plaza,East Rutherford,NJ,07073
5,1562,AMERICAN SOFTWARE -CL A,"470 East Paces Ferry Road, NE",Atlanta,GA,30305


In [148]:
chq.shape

(6250, 6)

In [149]:
chq.to_csv("../../data/chq.csv")

Now subset all the supply chain data to focus on transactions between companies in the U.S.  

In [152]:
c_linksUS = c_links[(c_links.gvkey.isin(chq.gvkey)) & (c_links.cgvkey.isin(chq.gvkey))]
c_linksUS.shape[0]/c_links.shape[0]

0.7294002994265699

In [153]:
c_links.head()

,gvkey,conm,cgvkey,cconm,cnms,srcdate,cid,sid,ctype,salecs,scusip,stic,ccusip,ctic,year
70,1013,ADC TELECOMMUNICATIONS INC,2136,VERIZON COMMUNICATIONS INC,VERIZON COMMUNICATIONS,20021031,10,0,COMPANY,111.056,000886309,ADCT.1,92343V104,VZ,2002
71,1013,ADC TELECOMMUNICATIONS INC,2136,VERIZON COMMUNICATIONS INC,VERIZON COMMUNICATIONS,20041031,13,0,COMPANY,104.312,000886309,ADCT.1,92343V104,VZ,2004
72,1013,ADC TELECOMMUNICATIONS INC,2136,VERIZON COMMUNICATIONS INC,VERIZON COMMUNICATIONS,20051031,13,0,COMPANY,146.000,000886309,ADCT.1,92343V104,VZ,2005
73,1013,ADC TELECOMMUNICATIONS INC,2136,VERIZON COMMUNICATIONS INC,VERIZON COMMUNICATIONS,20061031,13,0,COMPANY,205.000,000886309,ADCT.1,92343V104,VZ,2006
74,1013,ADC TELECOMMUNICATIONS INC,2136,VERIZON COMMUNICATIONS INC,VERIZON COMMUNICATIONS,20071031,13,0,COMPANY,236.000,000886309,ADCT.1,92343V104,VZ,2007


In [154]:
c_linksUS.shape

(51643, 15)

In [155]:
relevantUSgvkey = c_linksUS.gvkey.append(c_linksUS.cgvkey).drop_duplicates()

len(relevantUSgvkey)

5892

In [156]:
c_linksUS.to_csv("../../data/companyData/c_linksUS.csv")

## 2. Basic Change Information

In [52]:
compustatLast = compustat.copy()

compustatLast['year'] += 1
compustatLast.rename(columns = {'assets': 'assetsLast',
                                'netIncome':'netIncomeLast',
                                'totalRevenue':'totalRevenueLast',
                                'costGoodsSold':'costGoodsSoldLast',
                                'totalInv':'totalInvLast',
                                'opInc_afDep': 'opInc_afDepLast',
                                'opInc_befDep': 'opInc_befDepLast',
                                'priceClose': 'priceCloseLast',
                                'cash': 'cashLast'
                           }, inplace = True)

# 'assets','costGoodsSold', 'totalInv', 'netIncome', 'totalRevenue',
# 'opInc_afDep', 'opInc_befDep','priceClose','cash'

print(compustatLast.columns)
compustatLast = compustatLast[['gvkey','year','qtr','assetsLast','netIncomeLast',
                               'totalRevenueLast','costGoodsSoldLast','totalInvLast',
                              'opInc_afDepLast','opInc_befDepLast','priceCloseLast','cashLast']]

Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq',
       'assetsLast', 'cashLast', 'costGoodsSoldLast', 'totalInvLast',
       'netIncomeLast', 'opInc_afDepLast', 'opInc_befDepLast',
       'totalRevenueLast', 'costat', 'priceCloseLast', 'add1', 'addzip',
       'city', 'state'],
      dtype='object')


In [53]:
compustatLast.head()

,gvkey,year,qtr,assetsLast,netIncomeLast,totalRevenueLast,costGoodsSoldLast,totalInvLast,opInc_afDepLast,opInc_befDepLast,priceCloseLast,cashLast
0,1004,1999,3.0,1075.103596,15.602420,381.003873,309.704548,419.419406,28.748651,35.218539,22.960362,NaN
1,1082,1999,3.0,168.611793,-1.294889,42.417087,36.310010,17.313252,1.206842,2.371179,10.436528,NaN
2,1244,1999,3.0,27.689058,-2.324120,3.461132,1.621267,4.513134,-0.136624,-0.112335,23.529628,NaN
3,1258,1999,3.0,1.316141,1.217468,1.228095,0.000000,0.000000,1.217468,1.218987,NaN,NaN
4,1331,1999,3.0,3716.041664,0.593554,495.965079,NaN,677.551564,NaN,NaN,21.157689,NaN


In [54]:
compustatChanges = compustat.merge(compustatLast)
print(compustatChanges.shape)

(891345, 29)


In [19]:
compustatChanges['incomeChange']       = (compustatChanges.netIncome - compustatChanges.netIncomeLast)/compustatChanges.netIncomeLast

compustatChanges['revenueChange']      = (compustatChanges.totalRevenue - compustatChanges.totalRevenueLast)/compustatChanges.totalRevenueLast

compustatChanges['costChange']         = (compustatChanges.costGoodsSold - compustatChanges.costGoodsSoldLast)/compustatChanges.costGoodsSoldLast

compustatChanges['inventoryChange']    = (compustatChanges.totalInv - compustatChanges.totalInvLast)/compustatChanges.totalInvLast

compustatChanges['opInc_afDepChange']  = (compustatChanges.opInc_afDep - compustatChanges.opInc_afDepLast)/compustatChanges.opInc_afDepLast

compustatChanges['opInc_befDepChange'] = (compustatChanges.opInc_befDep - compustatChanges.opInc_befDepLast)/compustatChanges.opInc_befDepLast

compustatChanges['priceCloseChange']   = (compustatChanges.priceClose - compustatChanges.priceCloseLast)/compustatChanges.priceCloseLast

compustatChanges['assetsPrev']         = compustatChanges.assetsLast



print(compustatChanges.head())


'''compustatChanges = compustatChanges[['year', 'qtr', 'gvkey', 'companyName', 
                                     'tic', 'curcdq','salesChange','incomeChange','revenueChange',
                                     'costChange','inventoryChange']]
'''

   gvkey  datadate  year  qtr       companyName curcdq       assets  cash  \
0   1004  20000229  1999  3.0          AAR CORP    USD  1129.147545   NaN   
1   1082  20000131  1999  3.0     SERVIDYNE INC    USD   165.794462   NaN   
2   1244  20000229  1999  3.0       ALCIDE CORP    USD    21.983589   NaN   
3   1258  20000229  1999  3.0  CAPCO ENERGY INC    USD    41.520895   NaN   
4   1331  20000131  1999  3.0  ALPINE GROUP INC    USD          NaN   NaN   

   costGoodsSold    totalInv  ...  priceCloseLast  cashLast  incomeChange  \
0     333.569319  477.771178  ...       22.960362       NaN      0.051819   
1      39.497057   17.258803  ...       10.436528       NaN      2.519227   
2       2.067281    2.405836  ...       23.529628       NaN     -0.962616   
3       3.947309    1.427623  ...             NaN       NaN     -1.141501   
4            NaN         NaN  ...       21.157689       NaN           NaN   

   revenueChange costChange  inventoryChange opInc_afDepChange  \
0       

"compustatChanges = compustatChanges[['year', 'qtr', 'gvkey', 'companyName', \n                                     'tic', 'curcdq','salesChange','incomeChange','revenueChange',\n                                     'costChange','inventoryChange']]\n"

In [20]:
compustatChanges.to_csv("../../data/companyData/compustatChanges_all.csv")

In [21]:
compustatChanges.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,cash,costGoodsSold,totalInv,...,priceCloseLast,cashLast,incomeChange,revenueChange,costChange,inventoryChange,opInc_afDepChange,opInc_befDepChange,priceCloseChange,assetsPrev
0,1004,20000229,1999,3.0,AAR CORP,USD,1129.147545,NaN,333.569319,477.771178,...,22.960362,NaN,0.051819,0.070750,0.077057,0.139125,0.077069,0.074228,0.549549,1075.103596
1,1082,20000131,1999,3.0,SERVIDYNE INC,USD,165.794462,NaN,39.497057,17.258803,...,10.436528,NaN,2.519227,0.062572,0.087773,-0.003145,0.173010,-0.019499,-0.497620,168.611793
2,1244,20000229,1999,3.0,ALCIDE CORP,USD,21.983589,NaN,2.067281,2.405836,...,23.529628,NaN,-0.962616,0.244343,0.275102,-0.466926,0.666626,-2.373544,0.329021,27.689058
3,1258,20000229,1999,3.0,CAPCO ENERGY INC,USD,41.520895,NaN,3.947309,1.427623,...,NaN,NaN,-1.141501,3.207090,inf,inf,-1.249781,-1.245783,NaN,1.316141
4,1331,20000131,1999,3.0,ALPINE GROUP INC,USD,NaN,NaN,NaN,NaN,...,21.157689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.110536,3716.041664


## 3. Company-level controls

In [55]:
file = "../../data/companyData/compustatControls.csv"
compustatControls = pd.read_csv(file, encoding = 'unicode_escape')
compustatControls.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,datacqtr,datafqtr,atq,niq,costat,prccq,naics,sic
0,1000,19660331,1966,1.0,INDL,C,D,STD,AE.2,USD,1966Q1,1966Q1,NaN,NaN,I,NaN,NaN,3089.0
1,1000,19660630,1966,2.0,INDL,C,D,STD,AE.2,USD,1966Q2,1966Q2,NaN,NaN,I,NaN,NaN,3089.0
2,1000,19660930,1966,3.0,INDL,C,D,STD,AE.2,USD,1966Q3,1966Q3,NaN,NaN,I,NaN,NaN,3089.0
3,1000,19661231,1966,4.0,INDL,C,D,STD,AE.2,USD,1966Q4,1966Q4,NaN,0.164,I,NaN,NaN,3089.0
4,1000,19670331,1967,1.0,INDL,C,D,STD,AE.2,USD,1967Q1,1967Q1,NaN,NaN,I,NaN,NaN,3089.0


In [56]:
earliestYear = compustatControls.groupby('gvkey')['fyearq'].min().reset_index().\
    rename(columns = {'fyearq': 'earliestYear'})
earliestYear.head()

,gvkey,earliestYear
0,1000,1966
1,1001,1983
2,1003,1982
3,1004,1968
4,1005,1978


---------------------------------------

In [57]:
otherControls = compustatControls[['gvkey','fyearq','fqtr','sic','naics','atq','niq']]

In [58]:
defl.rename(columns = {'year': 'fyearq', 'qtr': 'fqtr'}, inplace = True)
defl.head()


otherControls = otherControls.merge(defl)


toDeflate = ['atq','niq']


for col in toDeflate:
    otherControls[col] = otherControls[col]*otherControls.inflator

otherControls.head()

,gvkey,fyearq,fqtr,sic,naics,atq,niq,inflator
0,1000,1966,1.0,3089.0,NaN,NaN,NaN,6.341516
1,1010,1966,1.0,3743.0,336510.0,NaN,43.306214,6.341516
2,1040,1966,1.0,3949.0,339920.0,NaN,42.830600,6.341516
3,1043,1966,1.0,5040.0,421410.0,NaN,29.507075,6.341516
4,1044,1966,1.0,2011.0,NaN,NaN,NaN,6.341516


In [59]:
otherControls['roa']  = otherControls['niq']/otherControls['atq']

otherControls.rename(columns = {'niq': 'netIncome',
                               'atq':  'assets'},inplace = True)

Try some industry classifications. First SIC2 and the corresponding groups.

In [60]:
otherControls = otherControls[~(otherControls['sic'] == 'na') & ~(otherControls['sic'].isna())]

otherControls['sic'] = otherControls['sic'].astype('int').astype('str').str.rjust(4,'0')

print(otherControls.sic.max(),otherControls.sic.min())

9998 0100


In [61]:
otherControls['sic2'] = otherControls.sic.str.slice(0,2).astype('int')

otherControls['sic']  = otherControls.sic.astype('int')

In [62]:
otherControls.loc[(otherControls.sic2  < 10 ), 'indGroup']                              = 'agForFish'
otherControls.loc[(otherControls.sic2 >= 10 ) & (otherControls.sic2 < 15 ) , 'indGroup'] = 'mining'
otherControls.loc[(otherControls.sic2 >= 15 ) & (otherControls.sic2 < 18 ) , 'indGroup'] = 'construction'
otherControls.loc[(otherControls.sic2 >= 20 ) & (otherControls.sic2 < 40 ) , 'indGroup'] = 'manu'
otherControls.loc[(otherControls.sic2 >= 40 ) & (otherControls.sic2 < 50 ) , 'indGroup'] = 'transportUtilities'
otherControls.loc[(otherControls.sic2 >= 50 ) & (otherControls.sic2 < 52 ) , 'indGroup'] = 'wholesale'
otherControls.loc[(otherControls.sic2 >= 52 ) & (otherControls.sic2 < 60 ) , 'indGroup'] = 'retail'
otherControls.loc[(otherControls.sic2 >= 60 ) & (otherControls.sic2 < 68 ) , 'indGroup'] = 'finance'
otherControls.loc[(otherControls.sic2 >= 70 ) & (otherControls.sic2 < 90 ) , 'indGroup'] = 'services'
otherControls.loc[(otherControls.sic2 >= 91 ) & (otherControls.sic2 < 98 ) , 'indGroup'] = 'publicAdmin'

In [63]:
otherControls.indGroup.value_counts()

manu                  617526
finance               474464
services              237786
transportUtilities    173488
mining                150302
retail                 88359
wholesale              53160
construction           17406
agForFish               6026
Name: indGroup, dtype: int64

In [65]:
otherControls.shape

(1845917, 11)

In previous versions of the code (pre-9/12), we also have the famafrench industries. But these seem to make distinctions that are so fine that they are not helpful, so they're out for now.

In [66]:
otherControls = otherControls[['gvkey','fyearq','fqtr','assets','netIncome','roa', 'sic2', 'indGroup']]

otherControls.rename(columns = {'assets': 'assetsLagged', 'netIncome': 'netIncomeLagged', 'roa': 'roa_lagged'}, 
                    inplace = True)

otherControls['year_toMatchOn'] = otherControls['fyearq'] + 2 

otherControls = otherControls.merge(earliestYear)

otherControls.head()

,gvkey,fyearq,fqtr,assetsLagged,netIncomeLagged,roa_lagged,sic2,indGroup,year_toMatchOn,earliestYear
0,1000,1966,1.0,NaN,NaN,NaN,30,manu,1968,1966
1,1000,1966,2.0,NaN,NaN,NaN,30,manu,1968,1966
2,1000,1966,3.0,NaN,NaN,NaN,30,manu,1968,1966
3,1000,1966,4.0,NaN,1.013227,NaN,30,manu,1968,1966
4,1000,1967,1.0,NaN,NaN,NaN,30,manu,1969,1966


In [67]:
otherControls = otherControls[otherControls.year_toMatchOn > 1997]

otherControls['ageTercile']  = pd.qcut(otherControls['earliestYear'], 3, labels=False)
otherControls['sizeTercile'] = pd.qcut(otherControls['assetsLagged'], 3, labels=False)
otherControls['profitTercile'] = pd.qcut(otherControls['roa_lagged'], 3, labels=False)

otherControls.profitTercile.value_counts()

2.0    300844
0.0    300844
1.0    300843
Name: profitTercile, dtype: int64

In [68]:
otherControls.earliestYear.max()

2020

In [36]:
otherControls.to_csv('../../data/companyData/otherControls.csv')

Save the basic industry data as well.

In [37]:
industries = otherControls[['gvkey','indGroup']].drop_duplicates()
industries.to_csv("../../data/companyData/gvkeyIndustries.csv")

# Final Merge

In [74]:
changes       = compustatChanges.copy()

otherControls = otherControls.rename(columns = {'year_toMatchOn': 'year','fqtr': 'qtr'})

In [75]:
otherControls.head()

,gvkey,fyearq,qtr,assetsLagged,netIncomeLagged,roa_lagged,sic2,indGroup,year,earliestYear,ageTercile,sizeTercile,profitTercile
169,1010,1996,1.0,3422.936770,46.757990,0.013660,37,manu,1998,1962,0,2.0,2.0
170,1010,1996,2.0,3454.132166,18.500640,0.005356,37,manu,1998,1962,0,2.0,1.0
171,1010,1996,3.0,3487.709411,10.470317,0.003002,37,manu,1998,1962,0,2.0,1.0
172,1010,1996,4.0,3448.667709,68.705992,0.019922,37,manu,1998,1962,0,2.0,2.0
173,1010,1997,1.0,4322.135119,260.987674,0.060384,37,manu,1999,1962,0,2.0,2.0


In [76]:
changes.columns

Index(['gvkey', 'datadate', 'year', 'qtr', 'companyName', 'curcdq', 'assets',
       'cash', 'costGoodsSold', 'totalInv', 'netIncome', 'opInc_afDep',
       'opInc_befDep', 'totalRevenue', 'costat', 'priceClose', 'add1',
       'addzip', 'city', 'state', 'assetsLast', 'netIncomeLast',
       'totalRevenueLast', 'costGoodsSoldLast', 'totalInvLast',
       'opInc_afDepLast', 'opInc_befDepLast', 'priceCloseLast', 'cashLast'],
      dtype='object')

In [77]:
changes = changes.merge(otherControls)

In [78]:
changes.shape

(827747, 39)

Put in calendar and fiscal quarters.

In [80]:
quarters = pd.read_csv("../../data/companyData/fiscalYears.csv")
quarters.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,curcdq,datacqtr,datafqtr,costat,fyrc
0,1004,19980228,1997,3.0,5,INDL,C,D,STD,AIR,USD,1998Q1,1997Q3,A,5
1,1004,19980531,1997,4.0,5,INDL,C,D,STD,AIR,USD,1998Q2,1997Q4,A,5
2,1004,19980831,1998,1.0,5,INDL,C,D,STD,AIR,USD,1998Q3,1998Q1,A,5
3,1004,19981130,1998,2.0,5,INDL,C,D,STD,AIR,USD,1998Q4,1998Q2,A,5
4,1004,19990228,1998,3.0,5,INDL,C,D,STD,AIR,USD,1999Q1,1998Q3,A,5


In [81]:
len(quarters.gvkey.unique())

27703

In [82]:
sum((quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9))/quarters.shape[0]

0.8661478661478661

In [83]:
quarters = quarters[(quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9)][['gvkey','datadate','datacqtr','datafqtr','fyr']].reset_index(drop = True)


In [84]:
quarters.head()

,gvkey,datadate,datacqtr,datafqtr,fyr
0,1010,19980331,1998Q1,1998Q1,12
1,1010,19980630,1998Q2,1998Q2,12
2,1010,19980930,1998Q3,1998Q3,12
3,1010,19981231,1998Q4,1998Q4,12
4,1010,19990331,1999Q1,1999Q1,12


Merge the quarter data into the change data, and make sure that the quarters that are used line up with the calendar quarters.

In [85]:
changesCal = changes[changes.gvkey.isin(quarters.gvkey.unique())]

changesCal = changesCal.merge(quarters)

print(changesCal.shape[0]/changes.shape[0])

0.7815951009185174


In [86]:
changesCal.loc[~(changesCal.datacqtr.isna()), 'year'] = changesCal.datacqtr.str.slice(0,4)
changesCal.loc[~(changesCal.datacqtr.isna()), 'qtr']  = changesCal.datacqtr.str.slice(5,6)

changesCal['DATE'] = pd.to_datetime(changesCal['datadate'])

changesCal.loc[(changesCal.datacqtr.isna()), 'year'] = changesCal.DATE.dt.year
changesCal.loc[(changesCal.datacqtr.isna()), 'qtr']  = changesCal.DATE.dt.quarter

changesCal['year'] = changesCal.year.astype('int64')
changesCal['qtr']  = changesCal.qtr.astype('int64')

print(changesCal.shape,changesCal.head())

(646963, 43)    gvkey  datadate  year  qtr              companyName curcdq      assets  \
0   8515  19990930  1999    3                  PHI INC    USD         NaN   
1  12405  19990930  1999    3               AVESIS INC    USD         NaN   
2  24474  19990930  1999    3  TECHNOLOGY SOLUTIONS CO    USD  362.960666   
3  26830  19990930  1999    3   NAMIBIAN MINERALS CORP    USD  111.299129   
4  30448  19990331  1970    1   AMERICAN HOMESTAR CORP    USD         NaN   

   cash  costGoodsSold  totalInv  ...  sic2            indGroup  earliestYear  \
0   NaN            NaN       NaN  ...    45  transportUtilities          1981   
1   NaN       2.470251       NaN  ...    64             finance          1984   
2   NaN      36.229854  0.000000  ...    73            services          1990   
3   NaN       7.938061  4.269385  ...    14              mining          1994   
4   NaN            NaN       NaN  ...    24                manu          1993   

   ageTercile sizeTercile  profitTerc

In [87]:
changesCal.to_csv("../../data/companyData/compustatChanges_withControls.csv")
changesCal.head()

,gvkey,datadate,year,qtr,companyName,curcdq,assets,cash,costGoodsSold,totalInv,...,sic2,indGroup,earliestYear,ageTercile,sizeTercile,profitTercile,datacqtr,datafqtr,fyr,DATE
0,8515,19990930,1999,3,PHI INC,USD,NaN,NaN,NaN,NaN,...,45,transportUtilities,1981,0,1.0,1.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
1,12405,19990930,1999,3,AVESIS INC,USD,NaN,NaN,2.470251,NaN,...,64,finance,1984,0,0.0,2.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
2,24474,19990930,1999,3,TECHNOLOGY SOLUTIONS CO,USD,362.960666,NaN,36.229854,0.000000,...,73,services,1990,0,1.0,2.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
3,26830,19990930,1999,3,NAMIBIAN MINERALS CORP,USD,111.299129,NaN,7.938061,4.269385,...,14,mining,1994,1,0.0,0.0,1999Q3,1999Q3,12,1970-01-01 00:00:00.019990930
4,30448,19990331,1970,1,AMERICAN HOMESTAR CORP,USD,NaN,NaN,NaN,NaN,...,24,manu,1993,0,1.0,2.0,NaN,1999Q3,6,1970-01-01 00:00:00.019990331
